In [ ]:
import pandas as pd
import os
import numpy as np

import requests
from urllib.parse import urlparse

In [ ]:
os.getcwd()

In [ ]:
df_list = list(os.listdir('data/parkinglot'))
df_list

----

<center><h2><b>1. 주차장 데이터 전처리</b></h2></center>

In [ ]:
# 데이터 파일 병합
# PATH = 'G:/Repository/PublicData/Jingu-Project/Parking/data/parkinglot'
# os.chdir(PATH)
# os.getcwd() # 현재 위치
# df_list = list(os.listdir(os.getcwd()))
df_list = list(os.listdir('data/parkinglot'))

df_merge = pd.DataFrame()
for d in df_list:
    df = pd.read_csv(f'./data/parkinglot/{d}', encoding='euc-kr')
    df_merge = pd.concat([df_merge, df], ignore_index=True)

df_merge.info()

In [ ]:
df_merge.head()

In [ ]:
df_merge['주차장구분'].unique()

In [ ]:
df = df_merge[['주차장명', '주차장구분', '주차장유형', '소재지도로명주소', '소재지지번주소', '주차구획수']]
df.head()

In [ ]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK APIKEY"})
    json_obj = result.json()
    keys = [key for key in json_obj]
    # print(json_obj[keys[0]])
    if keys[0] == 'documents':
        for document in json_obj['documents']:
            if document['address'] is not None:
                val = [document['address_name'], document['address']['region_3depth_h_name'], document['y'], document['x']]
            elif document['road_address'] is not None:
                val = [document['address_name'], document['road_address']['region_3depth_name'], document['y'], document['x']]
            return val
        return ["", "", "", ""]
    else:
        return ["", "", "", ""]

In [ ]:
df.loc[10:11]

In [ ]:
address = '부전동 474-17'
tmp = address_to_latlon(address)
tmp

In [ ]:
address = '부산광역시 부산진구 가야대로750번길'
tmp = address_to_latlon(address)
tmp

In [ ]:
df['주소'] = np.nan
df['행정동'] = np.nan
df['위도'] = np.nan
df['경도'] = np.nan
df.head()

In [ ]:
df[df['소재지도로명주소'].isnull()].head()

In [ ]:
df.loc[42:43]

In [ ]:
df['소재지도로명주소'][15] is not np.nan

In [ ]:
# 주소 넣기
for i in range(len(df)):
    if df['소재지도로명주소'][i] is not np.nan:
        df['주소'][i] = df['소재지도로명주소'][i]
    else:
        if df['소재지지번주소'][i] is not np.nan:
            df['주소'][i] =  df['소재지지번주소'][i]
        else:
            df['주소'][i] =  ""

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
address = '부산광역시 부산진구 가야대로750번길'
tmp = address_to_latlon(address)
tmp

In [ ]:
# 위도, 경도 구하기
for i in range(len(df)):
    min_address = df['주소'][i]
    tmp = address_to_latlon(min_address)
    df['행정동'][i] = tmp[1]
    df['위도'][i] = tmp[2]
    df['경도'][i] = tmp[3]
    # print(i)

In [ ]:
df.head()
df.info()

In [ ]:
df.to_csv('./data/result/주차장.csv', index=False, encoding='euc-kr')

In [ ]:
df_ = pd.read_csv('./data/result/주차장.csv', encoding='euc-kr')
df_ = df_[['주차장명', '행정동', '주차구획수', '위도', '경도']]
df_.head()

----

<center><h2><b>2. 공동주택 데이터 전처리</b></h2></center>

In [ ]:
df_h = pd.read_csv('./data/부산광역시_부산진구_공동주택현황_20210708.csv', encoding='euc-kr')
df_h.head()

In [ ]:
df_h.columns

In [ ]:
df_h.info()

In [ ]:
h = df_h[['단지명', '구분', '소재지 도로명주소', '소재지 지번주소', ' 세대수 ']]
h.head()

In [ ]:
h.rename(columns={'소재지 도로명주소':'소재지도로명주소', '소재지 지번주소':'소재지지번주소', ' 세대수 ':'세대수'}, inplace=True)
h.head()

In [ ]:
h.info()

In [ ]:
h['행정동'] = np.nan
h['위도'] = np.nan
h['경도'] = np.nan

In [ ]:
h.head()

In [ ]:
# 위도, 경도 구하기
for i in range(len(h)):
    min_address = h['소재지도로명주소'][i]
    tmp = address_to_latlon(min_address)
    h['행정동'][i] = tmp[1]
    h['위도'][i] = tmp[2]
    h['경도'][i] = tmp[3]

In [ ]:
h.head()

In [ ]:
h.to_csv('./data/result/공동주택.csv', encoding='euc-kr', index=False)

In [ ]:
h_ = pd.read_csv('./data/result/공동주택.csv', encoding='euc-kr')
h_ = h[['단지명', '행정동', '세대수', '위도', '경도']]
h_.head()

----

<center><h2><b>3. 상가 데이터 전처리</b></h2></center>

In [ ]:
df_m = pd.read_csv('./data/상가현황(부산진구_20210930).csv', encoding='cp949')
df_m.head(2)

In [ ]:
df_m.head()

In [ ]:
df_m.columns

In [ ]:
df_m.info()

In [ ]:
m = df_m[['상호명', '상권업종대분류명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '법정동명', '위도', '경도']]
m.head()

In [ ]:
m.to_csv('./data/result/상가.csv', encoding='euc-kr', index=False)

In [ ]:
m = pd.read_csv('./data/result/상가.csv', encoding='euc-kr')
m_ = m[['상호명', '행정동명', '위도', '경도']]
m_.head()

----

<center><h2><b>4. 데이터 그룹핑</b></h2></center>

In [ ]:
df_.head()

In [ ]:
p1 = df_.groupby('행정동').count()
p1 = p1[['주차장명']]
p1.rename(columns={'주차장명':'주차장수'}, inplace=True)
p1

p2 = df_.groupby('행정동').sum()
p2 = p2[['주차구획수']].astype(int)
p2

In [ ]:
h_.head()

In [ ]:
h1 = h_.groupby('행정동').count()
h1 = h1[['단지명']]
h1.rename(columns={'단지명':'공동주택수'}, inplace=True)
h1 = h1.astype(int)
h1

h2 = h_.groupby('행정동').sum()
h2 = h2[['세대수']]
h2.rename(columns={'세대수':'공동주택세대수'}, inplace=True)
h2 = h2.astype(int)
h2

In [ ]:
m_.head()

In [ ]:
m1 = m_.groupby('행정동명', as_index=True).count()
m1 = m1[['상호명']]
m1.rename(columns={'상호명':'상가수'})
m1

In [ ]:
m1.reset_index(inplace=True)
m1.rename(columns={'행정동명':'행정동'}, inplace=True)
m1.set_index(keys='행정동', inplace=True)
m1

In [ ]:
merged = p1.join(p2)
merged

In [ ]:
merged = merged.join(h1)
merged

In [ ]:
merged = merged.join(h2)
merged

In [ ]:
merged = merged.join(m1)
merged

In [ ]:
merged = merged.sort_index()

In [ ]:
merged.to_csv('./data/result/행정동기준_변수.csv', encoding='euc-kr', index=True)

----

<center><h2><b>5. 민원인 주소 데이터 전처리</b></h2></center>

In [ ]:
min_s = pd.read_csv("./data/민원/민원내용_생활불편신고_20190101-20201231.csv", encoding='euc-kr')
min_s.head()

In [ ]:
min_s.head()

In [ ]:
('불법주차' or '불법주정차' or '불법 주차' or '불법 주정차') in min_s.loc[0]['민원본내용']

In [ ]:
min_s_ = pd.DataFrame({'민원유형':[], '민원본내용':[], '민원발생주소':[]})
min_s_

In [ ]:
min_s.loc[0]['민원본내용']

In [ ]:
(min_s.loc[0]['민원본내용'].find('불법주차') != -1) or (min_s.loc[0]['민원본내용'].find('불법 주차') != -1) or (min_s.loc[0]['민원본내용'].find('불법주정차') != -1) or (min_s.loc[0]['민원본내용'].find('불법 주정차') != -1)

In [ ]:
min_s_ = min_s_.append(min_s.loc[0])
min_s_

In [ ]:
for i in range(len(min_s)):
    if (min_s.loc[i]['민원본내용'].find('불법주차') != -1) or (min_s.loc[i]['민원본내용'].find('불법 주차') != -1) or (min_s.loc[i]['민원본내용'].find('불법주정차') != -1) or (min_s.loc[i]['민원본내용'].find('불법 주정차') != -1):
        min_s_ = min_s_.append(min_s.loc[i])
    else:
        continue

In [ ]:
min_s_

In [ ]:
min_i = pd.read_csv('./data/민원/민원내용_이송이관_20190101-20201231.csv', encoding='euc-kr')
min_i

In [ ]:
min_i.loc[39118]

In [ ]:
min_i.loc[39118]['민원본내용']

In [ ]:
min_i.loc[39118]['민원본내용'].find('불법 주정차')

In [ ]:
min_i[min_i['민원본내용'].isnull()]

In [ ]:
min_i.dropna(inplace=True)

In [ ]:
min_i_ = pd.DataFrame({'민원유형':[], '민원본내용':[], '민원발생주소':[]})
min_i_

In [ ]:
min_i.info()

In [ ]:
min_i.reset_index(drop=True, inplace=True)

In [ ]:
min_i.info()

In [ ]:
min_i.head()

In [ ]:
min_si = min_s_
min_si.info()

In [ ]:
for i in range(len(min_i)):
    if (min_i.loc[i]['민원본내용'].find('불법주차') != -1) or (min_i.loc[i]['민원본내용'].find('불법 주차') != -1) or (min_i.loc[i]['민원본내용'].find('불법주정차') != -1) or (min_i.loc[i]['민원본내용'].find('불법 주정차') != -1):
        min_si = min_si.append(min_i.loc[i])
    else:
        continue

In [ ]:
min_si

In [ ]:
min_si.info()

In [ ]:
min_si.reset_index(drop=True, inplace=True)
min_si.info()

In [ ]:
min_si

In [ ]:
address = '부산광역시 부산진구 부전동 522-46 부근6'
tmp = address_to_latlon(address)
tmp

In [ ]:
min_s_.head()
min_s_.info()

In [ ]:
min_i_.head()

In [ ]:
df_s = pd.read_csv('./data/민원인주소_위도경도_원주소.csv', encoding='euc-kr')
df_s.head()

In [ ]:
df_s.head()

In [ ]:
df_s.dropna(inplace=True)
df_s.reset_index(inplace=True, drop=True)

In [ ]:
df_s.info()

In [ ]:
df_s.loc[0]

In [ ]:
address = '부산 부산진구 부전동 522-46'
tmp = address_to_latlon(address)
tmp

In [ ]:
min_si

In [ ]:
min_si['행정동'] = np.nan
min_si.head()

In [ ]:
min_si.loc[11:13]

In [ ]:
# 행정동 구하기
for i in range(len(min_si)):
    min_address = min_si['민원발생주소'][i]
    tmp = address_to_latlon(min_address)
    min_si['행정동'][i] = tmp[1]

In [ ]:
min_si[min_si['행정동'].isnull()]

In [ ]:
min_si2 = min_si[min_si['행정동'].isnull()]

In [ ]:
min_si2

In [ ]:
min_si2.reset_index(drop=True, inplace=True)
min_si2

In [ ]:
# 행정동 구하기
for i in range(len(min_si2)):
    min_address = min_si2['민원발생주소'][i]
    tmp = address_to_latlon(min_address)
    min_si2['행정동'][i] = tmp[1]

In [ ]:
min_si.loc[:21878]

In [ ]:
min_si2

In [ ]:
min_si_ = pd.concat([min_si.loc[:21878], min_si2])
min_si_

In [ ]:
min_si_.info()

In [ ]:
min_si_.reset_index(drop=True, inplace=True)
min_si_

In [ ]:
minwon = min_si_[['행정동', '민원유형']]
minwon = minwon.groupby('행정동').count()
minwon

In [ ]:
hh = pd.read_csv('./data/result/행정동기준_변수.csv', encoding='euc-kr')
hh.head()

In [ ]:
hh_ = hh.set_index('행정동')
hh_

In [ ]:
result = hh_.join(minwon)
result

In [ ]:
result.rename(columns={'상호명':'상가수', '민원유형':'민원수'}, inplace=True)
result

In [ ]:
result.to_csv('./data/result/행정동기준_결과.csv', encoding='euc-kr', index=True)